This notebook follows the case study example from https://github.com/cybera/case-study-example/blob/main/EDA_ookla_dataset.ipynb

In [ ]:
# To do: this is a test notebook

In [12]:
# Install and import modules

from google.colab import drive
drive.mount('/content/drive')

!pip install geopandas rtree pysal --quiet

import geopandas as gpd
import pandas as pd
import numpy as np
import rtree
from shapely.geometry import Polygon
import os
import pysal # PySAL: python Spatial Analysis Library
import mapclassify as mc
import matplotlib.pyplot as plt
import pickle

os.system('cls')

os.chdir('/content/drive/MyDrive/shared/dataset/')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
/content/drive/.shortcut-targets-by-id/1HoiIO5rDk02yAPQn_FwLi0Ry7lo64zYp/dataset


In [23]:
## Use Daisy's functions to aggregate data by quadkey

## !jupyter nbconvert --to python '/content/drive/MyDrive/shared/ssc22-case-comp/Daisy_dir/Exploratory_Data_Analysis.ipynb'

import sys
sys.path.append('/content/drive/MyDrive/shared/ssc22-case-comp/sonny_dir/')

from Exploratory_Data_Analysis import makeData, dataAggregation

In [25]:
mb_data = gdp.read_file('./Manitoba/Manitoba-speed-tiles.shp')
mb_agg = makeData(mb_data, 'quadkey', True, 'avg_d_mbps')
mb_agg.head()

conn_type          fixed                                                  \
Date             2019-Q1 2019-Q2 2019-Q3 2019-Q4 2020-Q1 2020-Q2 2020-Q3   
quadkey                                                                    
0211231312313313     NaN     NaN     NaN     NaN     NaN  10.305     NaN   
0211231330010202     NaN     NaN     NaN     NaN     NaN  30.669     NaN   
0211231332321100     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
0211322020122201     NaN     NaN     NaN   0.485     NaN     NaN     NaN   
0211322020122303     NaN     NaN   0.445     NaN     NaN     NaN   0.484   

conn_type                                 ...  mobile                          \
Date             2020-Q4 2021-Q1 2021-Q2  ... 2020-Q1 2020-Q2 2020-Q3 2020-Q4   
quadkey                                   ...                                   
0211231312313313     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN   
0211231330010202     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN   
0211231332321100     NaN     NaN     NaN  ...     NaN     NaN  14.863     NaN   
0211322020122201     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN   
0211322020122303     NaN    8.96   2.406  ...     NaN     NaN     NaN     NaN   

conn_type                                        tests  DA_POP  
Date             2021-Q1 2021-Q2 2021-Q3 2021-Q4                
quadkey                                                         
0211231312313313     NaN     NaN     NaN     NaN     2   405.0  
0211231330010202     NaN     NaN     NaN     NaN     1   405.0  
0211231332321100     NaN     NaN     NaN     NaN     1   405.0  
0211322020122201     NaN     NaN     NaN     NaN     2   990.0  
0211322020122303     NaN     NaN     NaN     NaN    12  1980.0  

[5 rows x 26 columns]

In [28]:
mb_agg.columns

MultiIndex([( 'fixed', '2019-Q1'),
            ( 'fixed', '2019-Q2'),
            ( 'fixed', '2019-Q3'),
            ( 'fixed', '2019-Q4'),
            ( 'fixed', '2020-Q1'),
            ( 'fixed', '2020-Q2'),
            ( 'fixed', '2020-Q3'),
            ( 'fixed', '2020-Q4'),
            ( 'fixed', '2021-Q1'),
            ( 'fixed', '2021-Q2'),
            ( 'fixed', '2021-Q3'),
            ( 'fixed', '2021-Q4'),
            ('mobile', '2019-Q1'),
            ('mobile', '2019-Q2'),
            ('mobile', '2019-Q3'),
            ('mobile', '2019-Q4'),
            ('mobile', '2020-Q1'),
            ('mobile', '2020-Q2'),
            ('mobile', '2020-Q3'),
            ('mobile', '2020-Q4'),
            ('mobile', '2021-Q1'),
            ('mobile', '2021-Q2'),
            ('mobile', '2021-Q3'),
            ('mobile', '2021-Q4'),
            ( 'tests',        ''),
            ('DA_POP',        '')],
           names=['conn_type', 'Date'])

In [40]:
print(mb_data['CDUID'].unique())
print(mb_data['CDNAME'].unique())

['4622' '4623' '4621' '4620' '4616' '4615' '4606' '4605' '4619' '4617'
 '4618' '4607' '4608' '4609' '4604' '4614' '4613' '4610' '4611' '4601'
 '4612' '4603' '4602']
['Division No. 22' 'Division No. 23' 'Division No. 21' 'Division No. 20'
 'Division No. 16' 'Division No. 15' 'Division No.  6' 'Division No.  5'
 'Division No. 19' 'Division No. 17' 'Division No. 18' 'Division No.  7'
 'Division No.  8' 'Division No.  9' 'Division No.  4' 'Division No. 14'
 'Division No. 13' 'Division No. 10' 'Division No. 11' 'Division No.  1'
 'Division No. 12' 'Division No.  3' 'Division No.  2']


In [37]:
yk_path = './Yukon/original/Yukon-speed-tiles.shp'
yk_data = gpd.read_file(yk_path)
print(yk_data['CDUID'].unique())
print(yk_data['CDNAME'].unique())
print(yk_data['PCTYPE'].unique())

['6001']
['Yukon']
[None '1']


In [39]:
nw_path = './Northwest Territories/Northwest Territories-speed-tiles.shp'
nw_data = gpd.read_file(nw_path)
print(nw_data['CDUID'].unique())
print(nw_data['CDNAME'].unique())
print(nw_data['PCTYPE'].unique())

['6101' '6102' '6103' '6104' '6106' '6105']
['Region 1' 'Region 2' 'Region 3' 'Region 4' 'Region 6' 'Region 5']
[None '4' '1']
